## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = list(range(1,1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark 
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

1

In [8]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [9]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [10]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [11]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([6.71496307e-01, 2.76345745e-01, 9.72966975e-01, 3.99447712e+00,
       3.13852971e+00, 5.39822021e+00, 5.17324206e+00, 4.74510781e+00,
       5.80267562e+00, 9.74066055e+00, 8.90393021e+00, 7.68216234e+00,
       4.71758090e+00, 2.02729660e+00, 1.43744361e+00, 7.19110181e+00,
       5.03853122e+00, 2.43061947e+00, 1.39648718e+01, 3.52075359e+00,
       1.44664436e+01, 2.19003980e+01, 9.50079690e+00, 2.17194131e+01,
       7.53155631e-02, 1.13729669e+01, 3.07064196e+00, 2.29341380e+00,
       1.78743796e+01, 2.70559965e+01, 1.21615439e+01, 3.09604447e+01,
       6.58644345e+00, 1.79135797e+00, 2.82896482e+01, 1.74938284e+01,
       2.19787797e+01, 8.83891879e+00, 3.77897367e+01, 3.52642377e+01,
       3.27228453e+01, 7.13276492e+00, 7.54082129e+00, 3.75481888e+01,
       2.95586473e+01, 2.65915058e+01, 6.86806715e+00, 2.85749701e+01,
       3.97930070e+01, 1.58982204e+01, 4.04209476e+01, 3.81310704e+01,
       2.23106388e+01, 3.70430172e+01, 3.21484975e+01, 1.85784552e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [12]:
price_items = sc.parallelize(sales_figures, 10)
price_items.take(4)

[0.671496307436765, 0.2763457453245943, 0.9729669747570651, 3.9944771156469665]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [13]:
def sales_tax(num):
    return num * (1 - 0.08)

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [14]:
# perform action to retrieve rdd values
revenue_minus_tax.take(4)

[0.6177766028418239,
 0.25423808569862677,
 0.8951296167764999,
 3.674918946395209]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [15]:
discounted = revenue_minus_tax.map(lambda x:x * (1 - 0.10))

In [16]:
discounted.take(10)

[0.5559989425576415,
 0.2288142771287641,
 0.8056166550988499,
 3.3074270517556883,
 2.598702600685449,
 4.469726333150979,
 4.2834444245706305,
 3.9289492677645486,
 4.8046154170847215,
 8.065266936288465]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [17]:
price_items.map(sales_tax).map(lambda x:x * (1 - 0.10)).top(15)

[794.1024851529062,
 778.815214660719,
 770.2233494709533,
 763.0435853239941,
 762.8022311444125,
 754.9167836676727,
 741.332661249561,
 723.390954918952,
 711.6086660121638,
 705.3571198540674,
 699.9518985292142,
 696.0229387035688,
 693.1784940443332,
 687.7533057325991,
 676.5267529057638]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [18]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:274 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [19]:
mapped = price_items.map(lambda x: (x, x * (1 - 0.08) * (1 - 0.10)))
print(mapped.count())
print(mapped.take(10))

1000
[(0.671496307436765, 0.5559989425576415), (0.2763457453245943, 0.2288142771287641), (0.9729669747570651, 0.8056166550988499), (3.9944771156469665, 3.3074270517556883), (3.1385297109727643, 2.598702600685449), (5.398220209119541, 4.469726333150979), (5.173242058660181, 4.2834444245706305), (4.745107811309841, 3.9289492677645486), (5.802675624498455, 4.8046154170847215), (9.740660551073024, 8.065266936288465)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [21]:
flat_mapped = price_items.flatMap(lambda x: (x, x * (1 - 0.08) * (1 - 0.10)))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.671496307436765, 0.5559989425576415, 0.2763457453245943, 0.2288142771287641, 0.9729669747570651, 0.8056166550988499, 3.9944771156469665, 3.3074270517556883, 3.1385297109727643, 2.598702600685449]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [22]:
# use the filter function
selected_items = discounted.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.count()

263

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [23]:
selected_items.reduce(lambda x, y: x + y)

120612.71131102399

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [24]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(28, 303.8023520320208),
 (12, 303.83998524283135),
 (13, 301.7686901675584),
 (7, 316.71617693649773),
 (21, 330.87807046651875),
 (1, 340.6218502646733),
 (22, 331.3376090362492)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [25]:
# calculate how much each user spent
sales_data.reduceByKey(lambda x, y: x + y)
sales_data.take(7)

[(18, 303.8023520320208),
 (4, 303.83998524283135),
 (2, 301.7686901675584),
 (35, 316.71617693649773),
 (33, 330.87807046651875),
 (8, 340.6218502646733),
 (48, 331.3376090362492)]

In [26]:
# sort the users from highest to lowest spenders
sales_data.reduceByKey(lambda x, y: x + y).sortBy(lambda x:x[1], ascending = False).collect()

[(8, 4650.254930111487),
 (28, 4351.285685271884),
 (27, 4340.156360909138),
 (36, 4185.361941188837),
 (39, 3989.6302717453873),
 (46, 3600.9156088214163),
 (5, 3561.635929246769),
 (29, 3540.309750896223),
 (37, 3488.792252158273),
 (25, 3416.503383304256),
 (33, 3247.916586403117),
 (42, 3223.801604931522),
 (44, 3140.572953305009),
 (19, 3088.7791676878483),
 (38, 3081.7967668169704),
 (13, 3035.498849672328),
 (48, 2784.619917074832),
 (32, 2716.587683543832),
 (20, 2684.003181452698),
 (31, 2636.2599824223416),
 (17, 2518.730298179158),
 (41, 2466.3698648795385),
 (43, 2375.3665255487026),
 (47, 2365.3338247623888),
 (2, 2273.539496975005),
 (1, 2247.3977249405684),
 (22, 2189.211025355379),
 (11, 2185.2390877078187),
 (35, 2134.9401438642208),
 (24, 2094.5286610864214),
 (3, 2084.5531182342015),
 (26, 2000.6235621116407),
 (12, 1916.587235207905),
 (45, 1915.8030474170885),
 (18, 1837.5682185582555),
 (34, 1808.1031579207204),
 (16, 1780.0849530759879),
 (50, 1651.87806411451),


Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [28]:
sorted(sales_data.countByKey().items(), key=lambda x:x[1], reverse=True)

[(50, 13),
 (44, 10),
 (28, 10),
 (4, 9),
 (6, 9),
 (7, 8),
 (48, 8),
 (42, 8),
 (22, 7),
 (36, 7),
 (16, 7),
 (23, 7),
 (35, 7),
 (9, 6),
 (25, 6),
 (8, 6),
 (32, 6),
 (41, 6),
 (33, 5),
 (26, 5),
 (47, 5),
 (1, 5),
 (3, 5),
 (45, 5),
 (5, 5),
 (13, 5),
 (20, 5),
 (24, 5),
 (39, 5),
 (30, 5),
 (15, 5),
 (19, 4),
 (2, 4),
 (29, 4),
 (21, 4),
 (46, 4),
 (27, 4),
 (14, 4),
 (40, 3),
 (31, 3),
 (12, 3),
 (10, 3),
 (49, 3),
 (11, 3),
 (37, 3),
 (38, 3),
 (34, 3),
 (43, 2),
 (17, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
